In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

/home/sasikala/yes/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  after removing the cwd from sys.path.


In [2]:
# pip install -U keras-tuner

In [3]:
df=pd.read_csv('Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [4]:
df.shape

(1093, 9)

In [5]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]


In [6]:
def build_model(hp):
    model=keras.Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        #select how may hidden layer we wnt
        model.add(layers.Dense(units=hp.Int('units_'+ str(i),
                                           min_value= 32,  #find how many neurons we wnt min,max
                                           max_value= 512,
                                           step=32),
                              activation='relu'))
        
    model.add(layers.Dense(1,activation='linear'))  #this is classification regression pbm ts y use linear
    model.compile(
            optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
            loss='mean_absolute_error',
            metrics=['mean_absolute_error'])
    return model

In [7]:
tuner=RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project1',
    project_name='Air quality index')   #running in 5times as 5 epochs ..with 3 iteration..
                                         # total 5*3 15 iterations

In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [9]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=0)

In [11]:
 tuner.search(xtrain,ytrain,epochs=5,validation_data=(xtest,ytest))

Trial 5 Complete [00h 00m 13s]
val_mean_absolute_error: 63.719340006510414

Best val_mean_absolute_error So Far: 56.956764221191406
Total elapsed time: 00h 00m 57s
INFO:tensorflow:Oracle triggered exit


In [12]:
tuner.results_summary()

Results summary
Results in project1/Air quality index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 8
units_0: 352
units_1: 256
learning_rate: 0.001
units_2: 256
units_3: 480
units_4: 416
units_5: 256
units_6: 480
units_7: 416
units_8: 480
units_9: 64
units_10: 224
units_11: 384
units_12: 64
Score: 56.956764221191406
Trial summary
Hyperparameters:
num_layers: 11
units_0: 96
units_1: 224
learning_rate: 0.001
units_2: 384
units_3: 128
units_4: 224
units_5: 96
units_6: 480
units_7: 512
units_8: 32
units_9: 160
units_10: 32
Score: 62.240133921305336
Trial summary
Hyperparameters:
num_layers: 13
units_0: 96
units_1: 256
learning_rate: 0.0001
units_2: 96
units_3: 288
units_4: 384
units_5: 288
units_6: 128
units_7: 192
units_8: 448
units_9: 352
units_10: 448
units_11: 32
units_12: 32
Score: 63.6212043762207
Trial summary
Hyperparameters:
num_layers: 17
units_0: 448
units_1: 160
learning_rate: 0.0001
units_2: 384
un